An entirely unsuccessful attempt at using a semi-supervised LDA for sentiment classification.

In [1]:
import os
import json
from sys import stdout
data_path = './datasets/fin_not_fin/'

In [2]:
def sweep_dir(folder, recursive=True, max_size=30_000):
    '''
    sweep a given folder for all images, process and load them into a np dataset which gets returned
    '''
    dataset = []
    for file in os.listdir(folder):
        if len(dataset) >= max_size:
            return dataset
        if file.endswith(".json"):
            with open(''.join([folder,file]), 'r') as fp:  
                dataset.append(json.load(fp))
        elif os.path.isdir(folder+file) and recursive is True: #if this is a directory
            dataset+= sweep_dir(folder+file+'/')
    return dataset
raw = sweep_dir(data_path+'financial/')

In [3]:
X = raw

In [4]:
from sklearn.pipeline import BaseEstimator, TransformerMixin, Pipeline
import urlextract
import re
class Cleaner(BaseEstimator, TransformerMixin):
    def __init__(self, include_subj=True, replace_html=False, remove_punctuation=True, replace_urls=False, replace_numbers=True):
        self.include_subj = include_subj; self.replace_html = replace_html
        self.remove_punctuation = remove_punctuation; self.replace_urls = replace_urls;
        self.replace_numbers = replace_numbers
    def fit(self, X):
        return self
    def transform(self, X):
        X_transformed = []
        for article in X:
            if type(article) is dict:
                text = " ".join([article['title'], article['text']]) if self.include_subj else article['text']
            elif type(article) is list:
                text = " ".join(article)
            else:
                text = article
            if self.replace_html:
                text = html_to_plain_text(text)
            if self.replace_urls:
                url_extractor = urlextract.URLExtract() 
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, " URL ")
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', ' NUMBER ', text)
            if self.remove_punctuation:
                text = text.replace("\'", "")
                text = text.replace("’", "")
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            X_transformed.append(text)
        return X_transformed

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
class CountVectorizerWithStemming(CountVectorizer):
    def build_analyzer(self):
        lemm = WordNetLemmatizer()
        analyzer = super(CountVectorizerWithStemming, self).build_analyzer()
        return lambda doc: (lemm.lemmatize(w) for w in analyzer(doc))

In [7]:
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer

cvws = CountVectorizerWithStemming(stop_words="english", max_features=20_000)
text_clf = Pipeline([
    ('clean', Cleaner()), #cleans text
    ('vect', cvws), #turns words to counts 
    #('tfidf', TfidfTransformer()), #turns counts to tf-idf
])
X_train_prepared = text_clf.fit_transform(X[:1000])
X_train_prepared

<1000x20000 sparse matrix of type '<class 'numpy.int64'>'
	with 166844 stored elements in Compressed Sparse Row format>

In [8]:
#seeding guided LDA
LDASeedList = [ ['jumped', 'hike', 'rise', 'growth', 'bullish', 'optimistic', 'rally', 'surge', \
                'soared', 'growth' 'buy', 'higher', 'gains', 'outperform','lower'],
                ['slumped', 'fell', 'worry', 'bearish', 'miss', 'sell', 'losses', 'warn', \
                 'plummet', 'bad', 'down', 'low', 'disappointed', 'weak', 'worry']
              ]

In [10]:
X_train_prepared

<1000x20000 sparse matrix of type '<class 'numpy.int64'>'
	with 166844 stored elements in Compressed Sparse Row format>

In [11]:
key_loc = {word:i for i,word in enumerate(cvws.get_feature_names())}
LDASeedList_prepared = {}
for t_id, st in enumerate(LDASeedList):
    for word in st:
        try:
            LDASeedList_prepared[key_loc[word]] = t_id
        except KeyError:
            continue

In [29]:
import guidedlda as LDA
#fix the hyperparams to be in the correct format and well be in business. Check CountVectorizer
#documentation to see how words in 
model = LDA.GuidedLDA(n_topics=2, n_iter=100, random_state=42, refresh=20)
model.fit(X_train_prepared, seed_topics= LDASeedList_prepared, seed_confidence=0.8)

INFO:guidedlda:n_documents: 1000
INFO:guidedlda:vocab_size: 20000
INFO:guidedlda:n_words: 291803
INFO:guidedlda:n_topics: 2
INFO:guidedlda:n_iter: 100
/anaconda3/lib/python3.6/site-packages/guidedlda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:guidedlda:<0> log likelihood: -2630843
INFO:guidedlda:<20> log likelihood: -2436418
INFO:guidedlda:<40> log likelihood: -2410220
INFO:guidedlda:<60> log likelihood: -2403653
INFO:guidedlda:<80> log likelihood: -2400710
INFO:guidedlda:<99> log likelihood: -2398679


In [30]:
n_top_words = 10
for i,topic_dist in enumerate(model.topic_word_):
    topic_words = np.array(cvws.get_feature_names())[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print("Topic {}: {} ".format(i,', '.join(topic_words)))

Topic 0: number, year, market, said, company, percent, bank, cent, share, price 
Topic 1: said, government, business, financial, fund, say, make, new, tax, people 


In [63]:
val = 8
LDAPredict(X_train_prepared[val], model.topic_word_)

[2.491167385522915, 0.20612463500845246]

In [53]:
model.topic_word_[3][key_loc['good']]

IndexError: index 3 is out of bounds for axis 0 with size 2

In [64]:
X[val]

{'organizations': [],
 'uuid': 'cb34de284d3c1c2434da1c3fe59e85e692f2fbfb',
 'thread': {'social': {'gplus': {'shares': 0},
   'pinterest': {'shares': 0},
   'vk': {'shares': 0},
   'linkedin': {'shares': 0},
   'facebook': {'likes': 0, 'shares': 0, 'comments': 0},
   'stumbledupon': {'shares': 0}},
  'site_full': 'business-news.thestreet.com',
  'main_image': '',
  'site_section': 'http://business-news.thestreet.com/ile-camera/rss/627',
  'section_title': 'Ile Camera : Personal Finance',
  'url': 'http://business-news.thestreet.com/ile-camera/story/can-you-protect-your-home-and-your-sanity-against-robo-calls/13208734',
  'country': 'US',
  'title': 'Can You Protect Your Home – and Your Sanity - Against Robo Calls?',
  'performance_score': 0,
  'site': 'thestreet.com',
  'participants_count': 0,
  'title_full': 'Can You Protect Your Home – and Your Sanity - Against Robo Calls?',
  'spam_score': 0.0,
  'site_type': 'news',
  'published': '2015-07-06T19:30:00.000+03:00',
  'replies_count':

In [70]:
import pickle
PATH = '../../ConsensusIO/dependencies/meta'
with open(PATH, 'rb') as fp:
    data_raw = pickle.load(fp)
X_test, y_test = data_raw["X"], np.array(data_raw["y"])
print(len(y_test))

1303


In [71]:
X_test_transformed = text_clf.transform(X_test)
X_test_transformed

<1303x20000 sparse matrix of type '<class 'numpy.int64'>'
	with 41115 stored elements in Compressed Sparse Row format>

In [78]:
X_test_yn = X_test_transformed[y_test!=1]
y_test_yn = y_test[y_test!=1]
y_test_yn.shape

(694,)

In [90]:
pred = [LDAPredict(article, model.topic_word_).index(max(LDAPredict(article, model.topic_word_))) for article in X_test_yn]


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


In [91]:
from sklearn.metrics import accuracy_score
accuracy_score(pred,y_test_yn)

0.42939481268011526

In [92]:
def LDAPredict(val, topic_word):
    return [sum(val*topic) for topic in topic_word]